# Лабораторная работа 2 по МО

Выбранный набор данных позволяет нам предсказать вероятность появления диабета у индийских пим в течение 5 лет, исходя из различных медицинских параметров.

Подключим необходимые библиотеки, подгрузим данные и удалим ненужные параметры

In [881]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import metrics  
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from numpy import log, dot, e
from numpy.random import rand


dataframe = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv', header=None)
dataframe.columns = ['Times pregnant', 'Glucose concentration', 'Blood pressure', 'Skinfold thickness', 'Serum insulin',
                     'BMI', 'Pedigree', 'Age', 'Class']
del dataframe['Skinfold thickness']
del dataframe['Serum insulin']
dataframe['Glucose concentration'] = dataframe['Glucose concentration'].replace(0, dataframe['Glucose concentration'].std())
dataframe['Blood pressure'] = dataframe['Blood pressure'].replace(0, dataframe['Blood pressure'].std())
dataframe['BMI'] = dataframe['BMI'].replace(0, dataframe['BMI'].std())



Вывод для атрибутов:

In [882]:
print(dataframe.head())
print("Размеры:",dataframe.shape)
print("Устройство:\n",dataframe.dtypes)

   Times pregnant  Glucose concentration  Blood pressure  ...  Pedigree  Age  Class
0               6                  148.0            72.0  ...     0.627   50      1
1               1                   85.0            66.0  ...     0.351   31      0
2               8                  183.0            64.0  ...     0.672   32      1
3               1                   89.0            66.0  ...     0.167   21      0
4               0                  137.0            40.0  ...     2.288   33      1

[5 rows x 7 columns]
Размеры: (768, 7)
Устройство:
 Times pregnant             int64
Glucose concentration    float64
Blood pressure           float64
BMI                      float64
Pedigree                 float64
Age                        int64
Class                      int64
dtype: object


Подгружаем данные и переводим их в удобный формат:

In [883]:
X = dataframe.iloc[:, :-1].values
Y = dataframe.iloc[:, 6].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,  random_state=0)
trans = StandardScaler()
X_train = trans.fit_transform(X_train)
X_test = trans.fit_transform(X_test)

####Логистическая регрессия

In [884]:
class MyLogisticRegression:
    def sigmoid(self, z): return 1 / (1 + e**(-z))
    
    def cost_function(self, X, y, weights):                 
        z = dot(X, weights)
        predict_1 = y * log(self.sigmoid(z))
        predict_0 = (1 - y) * log(1 - self.sigmoid(z))
        return -sum(predict_1 + predict_0) / len(X)
    
    def fit(self, X, y, epochs=25, lr=0.05):        
        loss = []
        weights = rand(X.shape[1])
        N = len(X)
                 
        for _ in range(epochs):        
            y_hat = self.sigmoid(dot(X, weights))
            weights -= lr * dot(X.T,  y_hat - y) / N            
            loss.append(self.cost_function(X, y, weights)) 
            
        self.weights = weights
        self.loss = loss
    
    def predict(self, X):        
        z = dot(X, self.weights)
        return [1 if i > 0.5 else 0 for i in self.sigmoid(z)]

Моя реализация

In [885]:
parameters = [(0.1, 50), (0.1, 100), (0.1, 1000), (0.1, 10000),
              (0.01, 50), (0.01, 100), (0.01, 1000), (0.01, 10000),
              (0.001, 50), (0.001, 100), (0.001, 1000), (0.001, 10000),
              (0.0001, 50), (0.0001, 100), (0.0001, 1000), (0.0001, 10000),
              (0.00001, 50), (0.00001, 100), (0.00001, 1000), (0.00001, 10000)
]
train_max = -1
test_max = -1
learning_rate_max = 0
epochs_max = 0
for i in parameters:
    learning_rate, epochs = i
    my_regr = MyLogisticRegression()
    my_regr.fit(X_train, Y_train, epochs, learning_rate)
    Y_pred_test = my_regr.predict(X_test)
    Y_pred_train = my_regr.predict(X_train)
    accuracy_train = accuracy_score(Y_train, Y_pred_train)
    accuracy_test = accuracy_score(Y_test, Y_pred_test)
    if (train_max < accuracy_train and test_max < accuracy_test):
      train_max = accuracy_train
      test_max = accuracy_test
      learning_rate_max = learning_rate
      epochs_max = epochs
print("Learning rate:", learning_rate, "epochs:", epochs, "\n", "Max train accuracy:", train_max, "Max test accuracy:", test_max)


Learning rate: 1e-05 epochs: 10000 
 Max train accuracy: 0.745928338762215 Max test accuracy: 0.7467532467532467


Лучше всего себя показала модель с learning_rate равным 0.00001 и количеством эпох равным 10000

In [886]:
my_regr = MyLogisticRegression()
my_regr.fit(X_train, Y_train, epochs = 10000, lr = 0.00001)
Y_pred = my_regr.predict(X_test)

Получим матрицу ошибок и метрики классификатора

In [887]:
print("confusion matrix:\n", confusion_matrix(Y_test, Y_pred))
print("accuracy:",accuracy_score(Y_test, Y_pred))
print("precision:",precision_score(Y_test, Y_pred))
print("recall:",recall_score(Y_test, Y_pred))
print("f1_score", f1_score(Y_test, Y_pred))

confusion matrix:
 [[75 32]
 [12 35]]
accuracy: 0.7142857142857143
precision: 0.5223880597014925
recall: 0.7446808510638298
f1_score 0.6140350877192982


Реализация sklearn

In [888]:
sk_regr = LogisticRegression().fit(X_train, Y_train)
Y_pred = sk_regr.predict(X_test)

Получим матрицу ошибок и метрики классификатора

In [889]:
print("confusion matrix:\n", confusion_matrix(Y_test, Y_pred))
print("accuracy:",accuracy_score(Y_test, Y_pred))
print("precision:",precision_score(Y_test, Y_pred))
print("recall:",recall_score(Y_test, Y_pred))
print("f1_score", f1_score(Y_test, Y_pred))

confusion matrix:
 [[94 13]
 [18 29]]
accuracy: 0.7987012987012987
precision: 0.6904761904761905
recall: 0.6170212765957447
f1_score 0.651685393258427


Получилась так, что в модели sklearn был оптимизирован параметр f1_score, что означает что модель попыталась достичь максимальной полноты и точности, а моя реализация наоборот получила максимальную полноту но меньшую точность.

####Дерево решений

In [890]:

class Node:
    def __init__(self, feature=None, threshold=None, data_left=None, data_right=None, gain=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.data_left = data_left
        self.data_right = data_right
        self.gain = gain
        self.value = value

class MyDecisionTree:
    def __init__(self, min_samples_split=2, max_depth=5):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None
        
    @staticmethod
    def _entropy(s):
        counts = np.bincount(np.array(s, dtype=np.int64))
        percentages = counts / len(s)
        entropy = 0
        for pct in percentages:
            if pct > 0:
                entropy += pct * np.log2(pct)
        return -entropy
    
    def _information_gain(self, parent, left_child, right_child):
        num_left = len(left_child) / len(parent)
        num_right = len(right_child) / len(parent)
        return self._entropy(parent) - (num_left * self._entropy(left_child) + num_right * self._entropy(right_child))
  
    def _best_split(self, X, y):
        best_split = {}
        best_info_gain = -1
        n_rows, n_cols = X.shape
 
        for f_idx in range(n_cols):
            X_curr = X[:, f_idx]

            for threshold in np.unique(X_curr):

                df = np.concatenate((X, y.reshape(1, -1).T), axis=1)
                df_left = np.array([row for row in df if row[f_idx] <= threshold])
                df_right = np.array([row for row in df if row[f_idx] > threshold])

                if len(df_left) > 0 and len(df_right) > 0:
                    y = df[:, -1]
                    y_left = df_left[:, -1]
                    y_right = df_right[:, -1]
                    gain = self._information_gain(y, y_left, y_right)
                    if gain > best_info_gain:
                        best_split = {
                            'feature_index': f_idx,
                            'threshold': threshold,
                            'df_left': df_left,
                            'df_right': df_right,
                            'gain': gain
                        }
                        best_info_gain = gain
        return best_split
    
    def _build(self, X, y, depth=0):
        n_rows, n_cols = X.shape
        
        if n_rows >= self.min_samples_split and depth <= self.max_depth:
            best = self._best_split(X, y)
            if best['gain'] > 0:
                left = self._build(
                    X=best['df_left'][:, :-1], 
                    y=best['df_left'][:, -1], 
                    depth=depth + 1
                )
                right = self._build(
                    X=best['df_right'][:, :-1], 
                    y=best['df_right'][:, -1], 
                    depth=depth + 1
                )
                return Node(
                    feature=best['feature_index'], 
                    threshold=best['threshold'], 
                    data_left=left, 
                    data_right=right, 
                    gain=best['gain']
                )
        return Node(
            value=Counter(y).most_common(1)[0][0]
        )
    
    def fit(self, X, y):
        self.root = self._build(X, y)
        
    def _predict(self, x, tree):
        if tree.value != None:
            return tree.value
        feature_value = x[tree.feature]

        if feature_value <= tree.threshold:
            return self._predict(x=x, tree=tree.data_left)
        
        if feature_value > tree.threshold:
            return self._predict(x=x, tree=tree.data_right)
        
    def predict(self, X):
        return [self._predict(x, self.root) for x in X]

Моя реализация

In [891]:
parameters = [(2, 1), (2, 2), (2, 3), (2, 10), (3, 15), 
              (3, 1), (3, 2), (3, 3), (3, 10), (3, 15), 
              (4, 1), (4, 2), (4, 3), (4, 10), (4, 15),
              (5, 1), (5, 2), (5, 3), (5, 10), (5, 15),
              (10, 1), (10, 2), (10, 3), (10, 10), (10, 15)
              ]
train_max = -1
test_max = -1
sample_split_max = 0
depth_max = 0
for i in parameters:
    sample_split, depth = i
    my_tree = MyDecisionTree(min_samples_split = sample_split, max_depth = depth)
    my_tree.fit(X_train, Y_train)
    Y_pred_test = my_tree.predict(X_test)
    Y_pred_train = my_tree.predict(X_train)
    accuracy_train = accuracy_score(Y_train, Y_pred_train)
    accuracy_test = accuracy_score(Y_test, Y_pred_test)
    if (train_max < accuracy_train and test_max < accuracy_test):
      train_max = accuracy_train
      test_max = accuracy_test
      sample_split_max = sample_split
      depth_max = depth 
print("Minimum sample splits:", sample_split_max, "Max depth:", depth_max, "\n", "Max train accuracy:", train_max, "Max test accuracy:", test_max)

Minimum sample splits: 2 Max depth: 1 
 Max train accuracy: 0.7638436482084691 Max test accuracy: 0.7597402597402597


Лучше всего себя показала модель с количеством образцов равным двум и максимальной глубиной равной 1

In [892]:
my_tree = MyDecisionTree(min_samples_split = 2, max_depth = 1)
my_tree.fit(X_train, Y_train)
Y_pred = my_tree.predict(X_test)

Получим матрицу ошибок и метрики классификатора

In [893]:
print("confusion matrix:\n", confusion_matrix(Y_test, Y_pred))
print("accuracy:",accuracy_score(Y_test, Y_pred))
print("precision:",precision_score(Y_test, Y_pred))
print("recall:",recall_score(Y_test, Y_pred))
print("f1_score", f1_score(Y_test, Y_pred))

confusion matrix:
 [[92 15]
 [22 25]]
accuracy: 0.7597402597402597
precision: 0.625
recall: 0.5319148936170213
f1_score 0.5747126436781609


Реализация sklearn

In [894]:
sk_tree = DecisionTree()
sk_tree.fit(X_train, Y_train)
Y_pred = sk_tree.predict(X_test)


Получим матрицу ошибок и метрики классификатора

In [895]:
print("confusion matrix:\n", confusion_matrix(Y_test, Y_pred))
print("accuracy:",accuracy_score(Y_test, Y_pred))
print("precision:",precision_score(Y_test, Y_pred))
print("recall:",recall_score(Y_test, Y_pred))
print("f1_score", f1_score(Y_test, Y_pred))

confusion matrix:
 [[94 13]
 [24 23]]
accuracy: 0.7597402597402597
precision: 0.6388888888888888
recall: 0.48936170212765956
f1_score 0.5542168674698795


Из-за особенностей нашего набора данных большинство метрик получились довольно низкими не считая простой вероятности, т.к. как можно заметить у нас очень мало положительных случаев true positive (23), что ведет к низкому recall и precision, то есть полноте и точности, и как следствие f1_score тоже низок.

####Random forest

In [896]:
class MyRandomForest:
    def __init__(self, num_trees=25, min_samples_split=2, max_depth=5):
        self.num_trees = num_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.decision_trees = []
        
    @staticmethod
    def _sample(X, y):
        n_rows, n_cols = X.shape
        samples = np.random.choice(a=n_rows, size=n_rows, replace=True)
        return X[samples], y[samples]
        
    def fit(self, X, y):
        if len(self.decision_trees) > 0:
            self.decision_trees = []
        num_built = 0
        while num_built < self.num_trees:
            try:
                clf = MyDecisionTree(
                    min_samples_split=self.min_samples_split,
                    max_depth=self.max_depth
                )
                _X, _y = self._sample(X, y)
                clf.fit(_X, _y)
                self.decision_trees.append(clf)
                num_built += 1
            except Exception as e:
                continue
    
    def predict(self, X):
        y = []
        for tree in self.decision_trees:
            y.append(tree.predict(X))
        y = np.swapaxes(a=y, axis1=0, axis2=1)
        predictions = []
        for preds in y:
            counter = Counter(preds)
            predictions.append(counter.most_common(1)[0][0])
        return predictions

Моя реализация

In [897]:
parameters = [(2, 2, 1), (5, 2, 1), (10, 2, 1), (20, 2, 1), (25, 2, 1)]
train_max = -1
test_max = -1
sample_split_max = 0
depth_max = 0
tree_count_max = 0
for i in parameters:
    tree_count, sample_split, depth = i
    my_forest = MyRandomForest(num_trees = tree_count, min_samples_split = sample_split, max_depth = depth)
    my_forest.fit(X_train, Y_train)
    Y_pred_test = my_forest.predict(X_test)
    Y_pred_train = my_forest.predict(X_train)
    accuracy_train = accuracy_score(Y_train, Y_pred_train)
    accuracy_test = accuracy_score(Y_test, Y_pred_test)
    if (train_max < accuracy_train and test_max < accuracy_test):
      train_max = accuracy_train
      test_max = accuracy_test
      sample_split_max = sample_split
      depth_max = depth
      tree_count_max = tree_count
print("Tree count:", tree_count_max, "Minimum sample splits:", sample_split_max, "Max depth:", depth_max, "\n", "Max train accuracy:", train_max, "Max test accuracy:", test_max)

Tree count: 10 Minimum sample splits: 2 Max depth: 1 
 Max train accuracy: 0.7719869706840391 Max test accuracy: 0.7727272727272727


Лучше всего себя показала модель с количеством деревьев равным 10

In [898]:
my_forest = MyRandomForest(num_trees = 10, min_samples_split = 2, max_depth = 1)
my_forest.fit(X_train, Y_train)
Y_pred = my_tree.predict(X_test)


Получим матрицу ошибок и метрики классификатора

In [899]:
print("confusion matrix:\n", confusion_matrix(Y_test, Y_pred))
print("accuracy:",accuracy_score(Y_test, Y_pred))
print("precision:",precision_score(Y_test, Y_pred))
print("recall:",recall_score(Y_test, Y_pred))
print("f1_score", f1_score(Y_test, Y_pred))

confusion matrix:
 [[92 15]
 [22 25]]
accuracy: 0.7597402597402597
precision: 0.625
recall: 0.5319148936170213
f1_score 0.5747126436781609


Реализация sklearn

In [900]:
sk_forest = RandomForestClassifier()
sk_forest.fit(X_train, Y_train)
Y_pred = sk_forest.predict(X_test)

Получим матрицу ошибок и метрики классификатора

In [901]:
print("confusion matrix:\n", confusion_matrix(Y_test, Y_pred))
print("accuracy:",accuracy_score(Y_test, Y_pred))
print("precision:",precision_score(Y_test, Y_pred))
print("recall:",recall_score(Y_test, Y_pred))
print("f1_score", f1_score(Y_test, Y_pred))

confusion matrix:
 [[92 15]
 [15 32]]
accuracy: 0.8051948051948052
precision: 0.6808510638297872
recall: 0.6808510638297872
f1_score 0.6808510638297872


В данном случае реализация sklearn значительно превзошла мою реализацию вероятнее всего в силу подборов оптимальных параметров, хотя вероятностная точность моей реализации всё ещё на достойном уровне.